# Mô tả bài toán
Trong các câu hỏi của phần **Text Classification**, **POS tagging**, chúng ta được cung cấp một tập dữ liệu nhỏ bao gồm hai chuỗi văn bản và các nhãn tương ứng trong đoạn code Python sau:

```python
corpus = [
    "you will get the low score",
    "more study more lucky come to you"
]
```
Quá trình tiền xử lý dữ liệu, xây dựng vocabulary, embedding được trực quan hóa như hình sau:

![image](https://firebasestorage.googleapis.com/v0/b/aivn-images.appspot.com/o/public%2F2025%2F3%2F2%2F1740886293065-image.png?alt=media&token=2a367e86-1eee-461e-b9ee-2fde92df5a42)

## Text Classification
Mục tiêu của bài toán này là xây dựng một mô hình phần loại text (0-Negative, 1-Positive) với Baseline cụ thể như hình sau:
![image](https://firebasestorage.googleapis.com/v0/b/aivn-images.appspot.com/o/public%2F2025%2F3%2F2%2F1740886064549-image.png?alt=media&token=f061b3b2-195d-484b-8a1b-db5a42c4b6aa)
Tất cả thông tin đều đã có ở trong phần mô tả, hãy đọc hiểu và trả lời các câu hỏi sau:

## Text Classification - CNN+Linear

In [ ]:
!pip install -U torchtext==0.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0

## Dataset

In [ ]:
import torch
import torch.nn as nn
# import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "you will get the low score",
    "more study more lucky come to you"
]
data_size = len(corpus)

# 0: negative - 1: positive
labels = [0, 1]

# Define the max vocabulary size and sequence length
vocab_size = 12
sequence_length = 6

In [ ]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'to': 11,
 'the': 10,
 'study': 9,
 'score': 8,
 'lucky': 7,
 'low': 6,
 'get': 5,
 'come': 4,
 'more': 2,
 '<pad>': 1,
 'you': 3,
 '<unk>': 0}

In [ ]:
# Tokenize and numericalize your samples
def vectorize(text, vocab, sequence_length):
    tokens = tokenizer(text)
    token_ids = [vocab[token] for token in tokens][:sequence_length]
    token_ids = token_ids + [vocab["<pad>"]] * (sequence_length - len(tokens))
    return torch.tensor(token_ids, dtype=torch.long)

# Vectorize the samples
corpus_ids = []
for sentence in corpus:
    corpus_ids.append(vectorize(sentence, vocab, sequence_length))

In [ ]:
for v in corpus_ids:
    print(v)

tensor([ 3,  0,  5, 10,  6,  8])
tensor([ 2,  9,  2,  7,  4, 11])


# Model

In [ ]:
class TCls_Model(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 2)
        custem_weight = torch.tensor([
            [ 0.26, -1.31],
            [ 0.72,  0.43],
            [-0.67,  0.61],
            [ 0.50,  0.50],
            [-0.26, -0.10],
            [ 1.29,  1.25],
            [ 1.95,  1.18],
            [-1.44, -1.89],
            [-0.20,  0.88],
            [-0.39,  1.07],
            [ 0.32, -0.05],
            [ 0.59, -0.98]
        ])
        self.embedding.weight = nn.Parameter(custem_weight)
        print((self.embedding.weight))
        self.conv1d = nn.Conv1d(2, 1, kernel_size=2)
        custem_conv_weight = torch.tensor([
            [[ 0.33, -0.26],
            [0.38, -0.46]]
        ])
        self.conv1d.weight = nn.Parameter(custem_conv_weight)
        custem_conv_weight = torch.tensor(
            [-0.30]
        )
        self.conv1d.bias = nn.Parameter(custem_conv_weight)

        print(self.conv1d.weight)
        print(self.conv1d.bias)


        self.fc = nn.Linear(5, num_classes)
        custem_fc_weight = torch.tensor([
          [ 0.40, -0.06, -0.44, -0.10,  0.43],
          [-0.19,  0.28, -0.04, -0.01,  0.23]
        ])
        self.fc.weight = nn.Parameter(custem_fc_weight)
        custem_fc_bias = torch.tensor(
            [0.18, 0.18]
        )
        self.fc.bias = nn.Parameter(custem_fc_bias)

        print(self.fc.weight)
        print(self.fc.bias)

        self.flatten = nn.Flatten()


    def forward(self, x):
        x = self.embedding(x)
        print(x.shape)
        x = x.permute(0, 2, 1)
        print(x.shape)
        x = self.conv1d(x)
        print(x.shape)
        print(f"Output conv1d: {x}")



        x = self.flatten(x)
        print(x.shape)
        x = self.fc(x)
        print(f"result: {x}")
        return x

num_classes = 2
model = TCls_Model(vocab_size, num_classes)

Parameter containing:
tensor([[ 0.2600, -1.3100],
        [ 0.7200,  0.4300],
        [-0.6700,  0.6100],
        [ 0.5000,  0.5000],
        [-0.2600, -0.1000],
        [ 1.2900,  1.2500],
        [ 1.9500,  1.1800],
        [-1.4400, -1.8900],
        [-0.2000,  0.8800],
        [-0.3900,  1.0700],
        [ 0.3200, -0.0500],
        [ 0.5900, -0.9800]], requires_grad=True)
Parameter containing:
tensor([[[ 0.3300, -0.2600],
         [ 0.3800, -0.4600]]], requires_grad=True)
Parameter containing:
tensor([-0.3000], requires_grad=True)
Parameter containing:
tensor([[ 0.4000, -0.0600, -0.4400, -0.1000,  0.4300],
        [-0.1900,  0.2800, -0.0400, -0.0100,  0.2300]], requires_grad=True)
Parameter containing:
tensor([0.1800, 0.1800], requires_grad=True)


# Forward input 1

In [ ]:
input_1 = torch.tensor([[3, 0, 5, 10, 6, 8]], dtype=torch.long)
label_1 = torch.tensor([0], dtype=torch.long)

output = model(input_1)
print(output.shape)

torch.Size([1, 6, 2])
torch.Size([1, 2, 6])
torch.Size([1, 1, 5])
Output conv1d: tensor([[[ 0.5900, -1.6224,  0.5405, -1.2632,  0.4391]]],
       grad_fn=<ConvolutionBackward0>)
torch.Size([1, 5])
result: tensor([[ 0.5907, -0.2944]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])


In [11]:
import torch
import torch.nn as nn

conv = torch.nn.Conv1d(2, 1, kernel_size=2)
conv.weight = torch.nn.Parameter(torch.tensor([[[0.33, -0.26], [0.38, -0.46]]]))
conv.bias = torch.nn.Parameter(torch.tensor([-0.30]))
embedding = nn.Embedding(12, 2)
embedding.weight = nn.Parameter(
    torch.tensor(
        [
            [0.26, -1.31],
            [0.72, 0.43],
            [-0.67, 0.61],
            [0.50, 0.50],
            [-0.26, -0.10],
            [1.29, 1.25],
            [1.95, 1.18],
            [-1.44, -1.89],
            [-0.20, 0.88],
            [-0.39, 1.07],
            [0.32, -0.05],
            [0.59, -0.98],
        ]
    )
)
input = torch.tensor([[3, 0, 5, 10, 6, 8]], dtype=torch.long)
input = embedding(input)
input = input.permute(0, 2, 1)
conv_out = conv(input)
conv_out

tensor([[[ 0.5900, -1.6224,  0.5405, -1.2632,  0.4391]]],
       grad_fn=<ConvolutionBackward0>)

## M08CLS01
### Câu hỏi
Hãy xác định shape của **Input Embedding** và shape đầu vào của **convolution model Conv1d** lần lượt là?  

A.
```
(1, 2, 6); (1, 6, 2)
```  
B.
```
(1, 6, 2); (1, 1, 2)
```
C.
```
(1, 6, 2); (1, 2, 6)
```
D.
```
(1, 2, 6); (1, 2, 1)
```

### Đáp án:
C (Trước khi đưa vào Conv1D cần permute)

## M08CLS02
### Câu hỏi
**Input** của **FC layer** gồm bao nhiêu node?.  
A. 3  
B. 4  
C. 5  
D. 6  
### Đáp án:
C: 5 (Sử dụng công thức Convolution để tính)


## M08CLS03
### Câu hỏi
Đâu là giá trị output của **conv1d** khi đưa **sample1** vào model (không quan trọng shape)?  
A.
```
[0.59, -1.62,  0.54, -1.26,  0.44]
```  
B.
```
[0.73, -1.45, 0.28, -1.89, 0.61]
```
C.
```
[-0.15, 1.27, -0.92, 0.83, -1.34]
```
D.
```
[1.12, -0.67, 0.95, -1.08, 0.39]
```

Đáp án: A

# Forward input 2

In [ ]:
input_2 = torch.tensor([[2, 9, 2, 7, 4, 11]], dtype=torch.long)
label_2 = torch.tensor([1], dtype=torch.long)

output = model(input_2)
print(output.shape)

torch.Size([1, 6, 2])
torch.Size([1, 2, 6])
torch.Size([1, 1, 5])
Output conv1d: tensor([[[-0.6801, -0.1285,  0.9545, -1.3798, -0.1264]]],
       grad_fn=<ConvolutionBackward0>)
torch.Size([1, 5])
result: tensor([[-0.4207,  0.2198]], grad_fn=<AddmmBackward0>)
torch.Size([1, 2])


# Sofmaxt output

In [ ]:
import numpy as np
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# Output của sample 1 và sample 2
output1 = np.array([0.5907, -0.2944])
output2 = np.array([-0.4207,  0.2198])

# Tính softmax
softmax1 = softmax(output1)
softmax2 = softmax(output2)

print("Softmax của sample 1:", softmax1)
print("Softmax của sample 2:", softmax2)

Softmax của sample 1: [0.70787795 0.29212205]
Softmax của sample 2: [0.34513352 0.65486648]


In [ ]:
softmax1[0] + softmax2[0]

1.0530114729878066